In [1]:
import xarray as xr
import numpy as np
import rioxarray
import geopandas as gpd
from ipyleaflet import Map, ImageOverlay, WidgetControl, LayersControl, basemaps, GeoJSON, TileLayer, Heatmap, Marker, Popup
import ipywidgets as widgets
from utils import leaflet_bounds, scalar_to_base64_image, find_intersections
import pandas as pd
from shapely.geometry import Point

In [2]:
# -------------------------------
# Data Paths
# -------------------------------
download_path = "./data_download"
intermediate_path = "./data_intermediate"
# terrain_path = f"{download_path}/data/cv_terrain.tiff"
vector_rivers = f"{download_path}/data/shp/cv_rivers.geojson"
vector_subbasin = f"{download_path}/data/shp/subbasins_cv_clip.geojson"
points_resistivity = f"{download_path}/data/aem/em_resistivity.csv"
metric_zarr_path = f"{intermediate_path}/consolidated_metric_output.zarr"


target_epsg = 4326
center = [37.66335291403956, -120.69523554193438]
zoom = 6
# basemap = basemaps.CartoDB.Positron 
basemap = None
map_width = '500px'
map_height = '800px'

In [3]:
# -------------------------------
# Data Loading
# -------------------------------

# Load vector layers with GeoPandas
rivers = gpd.read_file(vector_rivers)
rivers = rivers.to_crs(epsg=target_epsg)

def combine_rivers_gdf(river_gdf, name_column='GNIS_Name'):
    """
    Combine river fragments with the same name into single features.
    
    Parameters:
    -----------
    river_gdf : GeoDataFrame with river LineStrings
    name_column : str, column containing river names
    
    Returns:
    --------
    GeoDataFrame with dissolved geometries, 'name' column
    """
    rivers_combined = river_gdf.dissolve(by=name_column).reset_index()
    # rivers_combined = rivers_combined.rename(columns={name_column: 'name'})
    return rivers_combined[[name_column, 'geometry']]

rivers = combine_rivers_gdf(rivers)

subbasins = gpd.read_file(vector_subbasin)
subbasins = subbasins.to_crs(epsg=target_epsg)

# find outer basin intersections with rivers
river_intersections = find_intersections(rivers, subbasins)

df = pd.read_csv(points_resistivity)
# Create GeoDataFrame from the UTM coordinates
resistivity_profiles = gpd.GeoDataFrame(
    df,
    geometry=[Point(x, y) for x, y in zip(df['UTMX'], df['UTMY'])],
    crs='EPSG:3310'  # TODO: Confirm EPSG!!!
)
resistivity_profiles = resistivity_profiles.to_crs(f'EPSG:{target_epsg}')
# for now resample all points 
# TODO: Discuss better performance options
resistivity_profiles = resistivity_profiles.sample(10000)

# Load consolidated metric dataset
ds = xr.open_zarr(metric_zarr_path)
ds = ds.transpose('fraction', 'y', 'x')
ds = ds.sortby('y', ascending=False)
ds = ds.sortby('x', ascending=True)
ds.rio.write_crs(3310, inplace=True)
ds_reprojected = ds.rio.reproject(f"EPSG:{target_epsg}")

## TESTING: Simplify geometries before building layers to improve performance

# Simplify geometries (tolerance in degrees, ~0.001 = ~100m)
subbasins['geometry'] = subbasins.geometry.simplify(tolerance=0.001, preserve_topology=True)
rivers['geometry'] = rivers.geometry.simplify(tolerance=0.001, preserve_topology=True)


In [4]:
# Create layers
## Terrain context
# Ocean basemap (includes bathymetry)
l_ocean = TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}',
    name='Ocean/Water',
    opacity=1.0
)
# Hillshade with water areas
l_elevation = TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/Elevation/World_Hillshade/MapServer/tile/{z}/{y}/{x}',
    name='Hillshade',
    opacity=1.0
)

# Vector layers
l_rivers = GeoJSON(
    data=rivers.__geo_interface__,
    style={'color': 'blue', 'weight': 1, 'opacity': 0.7},
    name="Rivers"
)

l_subbasins = GeoJSON(
    data=subbasins.__geo_interface__,
    style={'color': 'black', 'weight': 1, 'fill':False},
    name="Subbasins"
)

# Marker layers
# Add intersection markers with tooltips
river_inflow_layers = []
for idx, row in river_intersections.iterrows():
    marker = Marker(
        location=(row.geometry.y, row.geometry.x),
        title=row['river_name'],
        draggable=False
    )
    popup = Popup(
        child=widgets.HTML(value=f"<b>{row['river_name']}</b>"),
        close_button=False,
        auto_close=False
    )
    marker.popup = popup
    river_inflow_layers.append(marker)

# intersections_layer = GeoJSON(
#     data=river_intersections.__geo_interface__,
#     point_style={'radius': 8, 'color': 'red', 'fillColor': 'orange', 'fillOpacity': 0.8},
#     name="River Exits"
# )

# Point layers
l_resistivity = GeoJSON(
    data=resistivity_profiles.__geo_interface__,
    point_style={
        'radius': 0.01,
        'color': 'red',
        'fillColor': 'red',
        'fillOpacity': 0.6,
        'weight': 0.1
    },
    name='Resistivity Profiles (subsampled)',
)

l_sediment = GeoJSON(
    data=resistivity_profiles.__geo_interface__,
    point_style={
        'radius': 0.01,
        'color': 'brown',
        'fillColor': 'brown',
        'fillOpacity': 0.6,
        'weight': 0.1
    },
    name='Sediment Type Profiles (subsampled)',
)

# Heatmap layers
resistivity_locations = [[point.y, point.x] for point in resistivity_profiles.geometry]
l_resistivity_heatmap = Heatmap(
    locations=resistivity_locations,
    radius=5,
    blur=2,
    name='Resistivity Heatmap (subsampled)'
)

l_sediment_heatmap = Heatmap(
    locations=resistivity_locations,  # Using same data for now
    radius=10,
    blur=2,
    gradient={0.4: 'blue', 0.6: 'cyan', 0.7: 'lime', 0.8: 'yellow', 1.0: 'red'},
    name='Sediment Heatmap (subsampled)'
)

In [5]:
#| label: interactive:fig-1

##########
# Figure 1
##########
m = Map(
    center=center, 
    zoom=zoom, 
    # basemap=basemap, #TODO fix
    scroll_wheel_zoom=True,
    layout=widgets.Layout(width=map_width, height=map_height)
)

# Dictionary mapping dropdown options to layer objects
layer_map = {layer.name: layer for layer in [
    l_resistivity,
    l_resistivity_heatmap,
    l_sediment,
    l_sediment_heatmap
]}
init_key = list(layer_map.keys())[0]

m.add_layer(l_ocean)
m.add_layer(l_elevation)
m.add_layer(l_rivers)
m.add_layer(l_subbasins)
m.add_layer(l_resistivity)

# Create dropdown to switch between layers
layer_dropdown = widgets.Dropdown(
    options=list(layer_map.keys()),
    value=init_key,
    description='Data Layer:',
    style={'description_width': 'initial'}
)

# Current active layer
current_layer = layer_map[init_key]

def on_layer_change(change):
    """Handle layer selection change"""
    global current_layer
    new_layer_name = change['new']
    new_layer = layer_map[new_layer_name]
    
    # Remove current layer and add new one
    m.remove_layer(current_layer)
    m.add_layer(new_layer)
    current_layer = new_layer

layer_dropdown.observe(on_layer_change, names='value')

# Add dropdown control to map
widget_control = WidgetControl(widget=layer_dropdown, position='topright')
m.add_control(widget_control)
m.add_control(LayersControl(position='topleft'))
m

Map(center=[37.66335291403956, -120.69523554193438], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [6]:
from ipyleaflet import Map, GeoJSON, WidgetControl, LayersControl, basemaps
import ipywidgets as widgets
import geopandas as gpd
from IPython.display import display
from ipywidgets import jslink

# Define the column name for subbasin selection
subbasin_column = "Basin_Su_1"

debug_output = widgets.Output()

# Create the maps
layout = widgets.Layout(width=map_width, height=map_height)
m1 = Map(
    center=[37.66335291403956, -120.69523554193438],
    zoom=6,
    basemap=basemaps.CartoDB.Positron,
    layout=layout
    )
m2 = Map(
    center=[37.66335291403956, -120.69523554193438],
    zoom=6, 
    basemap=basemaps.CartoDB.Positron,
    layout=layout
    )

# Sync the views between the two maps
jslink((m1, 'center'), (m2, 'center'))
jslink((m1, 'zoom'), (m2, 'zoom'))

# Add rivers
m1.add_layer(l_rivers)
m2.add_layer(l_rivers)

# Add river inflow markers
for marker in river_inflow_layers:
    m1.add_layer(marker)
    m2.add_layer(marker)

# Add the subbasins layer to both maps
m1.add_layer(l_subbasins)
m2.add_layer(l_subbasins)

# # TODO: factor out clims and names to the top
# clim_dict = {
#     'fraction_coarse':[0,100, name...]
# }

# Add raster data left panel
ds_reprojected
fcd_ave = ds_reprojected['fraction_coarse']
fcd_layer = ImageOverlay(
    url=scalar_to_base64_image(fcd_ave, cmap='RdBu_r',vmin=0,vmax=1), 
    bounds=leaflet_bounds(fcd_ave), 
    opacity=1.0, 
    name="Fraction Coarse Dominated [%]",
)
m1.add_layer(fcd_layer)

# Add raster data right panel
current_dataset_name = "path_length_norm"
da_scalar = ds_reprojected[current_dataset_name].sel(fraction=0.2).load()
# Add scalar data overlay (will be updated by threshold and dataset selection)
initial_scalar = scalar_to_base64_image(da_scalar, cmap='Greens')
scalar_overlay = ImageOverlay(
    url=initial_scalar, 
    bounds=leaflet_bounds(da_scalar), 
    opacity=1.0, 
    name="Scalar Data"
)
m2.add_layer(scalar_overlay)

# -------------------------------
# Controls: Dataset dropdown and threshold slider
# -------------------------------
def update_scalar_overlay():
    """Update the scalar overlay based on current dataset and threshold"""
    threshold = slider.value
    # da_masked = da_scalar.where(da_scalar >= threshold)
    if 'fraction' in da_scalar.dims:
        da_overlay = da_scalar.sel(fraction=threshold)
    else:
        da_overlay = da_scalar

    scalar_overlay.url = scalar_to_base64_image(
        da_overlay, 
        cmap='Greens',
        vmin=float(np.nanpercentile(da_scalar.values, 0.1)),
        vmax=float(np.nanpercentile(da_scalar.values, 0.9))
    )

def on_dataset_change(change):
    """Handle dataset selection change"""
    global da_scalar, current_dataset_name
    current_dataset_name = change['new']
    da_scalar = ds[current_dataset_name]
    
    # Update the overlay
    update_scalar_overlay()

def on_threshold_change(change):
    """Handle threshold slider change"""
    update_scalar_overlay()

# Dataset dropdown
dropdown = widgets.Dropdown(
    options=list([v for v in ds.data_vars if v not in ['fraction_coarse']]),
    value=current_dataset_name,
    description='Dataset:',
    style={'description_width': 'initial'}
)

dropdown.observe(on_dataset_change, names='value')

slider = widgets.SelectionSlider(
    options=ds.fraction.values,
    value=0.1,  # Must be one of the values in options
    description='FCD Threshold',
    style={'description_width': 'initial'}
)

slider.observe(on_threshold_change, names='value')

# Combine controls in a VBox
controls = widgets.VBox([dropdown, slider])
widget_control = WidgetControl(widget=controls, position='topright')
m2.add_control(widget_control)


# Create dropdown with "All Regions"
subbasin_names = ['All Regions'] + subbasins[subbasin_column].tolist()
dropdown = widgets.Dropdown(
    options=subbasin_names,
    description='Subbasin:',
    style={'description_width': 'initial'}
)

# Create highlight layer
highlight_layer = GeoJSON(
    data={},
    style={'color': 'orange', 'weight': 2, 'fillColor': 'orange', 'fillOpacity': 0.5},
    name="Highlighted Subbasin"
)
m1.add_layer(highlight_layer)
m2.add_layer(highlight_layer)

# Function to zoom to selected subbasin
def zoom_to_subbasin(change):
    global highlight_layer
    selected_name = change['new']
    
    if selected_name == 'All Regions':
        m1.center = [37.66335291403956, -120.69523554193438]
        m1.zoom = 6
        # m2 get synced
        highlight_layer.data = {}
    else:
        selected_subbasin = subbasins[subbasins[subbasin_column] == selected_name]
        
        if selected_subbasin.empty:
            print("No matching subbasin found!")
            return
        
        bounds = selected_subbasin.total_bounds
        m1.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])
        
        # Update both data AND style for both maps
        m1.remove_layer(highlight_layer)
        m2.remove_layer(highlight_layer)
        highlight_layer = GeoJSON(
            data=selected_subbasin.__geo_interface__,
            style={'color': 'orange', 'weight': 3, 'opacity': 0.5,
                    'fillColor': 'orange', 'fillOpacity': 0.0 # fill color interferes with the raster data (was nice for debugging)
                    },
            name="Highlighted Subbasin"
        )
        m1.add_layer(highlight_layer)
        m2.add_layer(highlight_layer)

# Attach observer
dropdown.observe(zoom_to_subbasin, names='value')

# Add dropdown control
control = WidgetControl(widget=dropdown, position='topright')
m1.add_control(control)

# Add layer controls
m1.add_control(LayersControl(position='bottomright'))
m2.add_control(LayersControl(position='bottomright'))

# Display debug output and maps
display(debug_output)
display(widgets.HBox([m1, m2]))

Output()